In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('cleaned.csv')

In [3]:
df

,sent_id,token_text,token_lemma,token_pos,token_tag,token_dep,token_shape,token_is_alpha,token_is_stop,NER_tags
0,0,1,1,NUM,CD,nummod,d,False,False,B-QUANTITY
1,0,slice,slice,NOUN,NN,nmod,xxxx,True,False,B-UNIT
2,0,whole,whole,ADJ,JJ,amod,xxxx,True,True,B-ING
3,0,wheat,wheat,NOUN,NN,compound,xxxx,True,False,I-ING
4,0,bread,bread,NOUN,NN,ROOT,xxxx,True,False,I-ING
...,...,...,...,...,...,...,...,...,...,...
1397955,181969,cut,cut,VERB,VBD,ROOT,xxx,True,False,B-STATE
1397956,181969,into,into,ADP,IN,prep,xxxx,True,True,I-STATE
1397957,181969,2,2,NUM,CD,nummod,d,False,False,I-STATE
1397958,181969,inch,inch,NOUN,NN,compound,xxxx,True,False,I-STATE


In [4]:
sents = []
temp = []
tags = []
temp_tags = []
current_id = 0

for item in df.itertuples(index=False):
    if item.sent_id != current_id:
        current_id += 1
        sents.append(temp)
        tags.append(temp_tags)
        temp = []
        temp_tags = []
    temp.append(item)
    temp_tags.append(item.NER_tags)


In [5]:
sents[:3]

[[Pandas(sent_id=0, token_text='1', token_lemma='1', token_pos='NUM', token_tag='CD', token_dep='nummod', token_shape='d', token_is_alpha=False, token_is_stop=False, NER_tags='B-QUANTITY'),
  Pandas(sent_id=0, token_text='slice', token_lemma='slice', token_pos='NOUN', token_tag='NN', token_dep='nmod', token_shape='xxxx', token_is_alpha=True, token_is_stop=False, NER_tags='B-UNIT'),
  Pandas(sent_id=0, token_text='whole', token_lemma='whole', token_pos='ADJ', token_tag='JJ', token_dep='amod', token_shape='xxxx', token_is_alpha=True, token_is_stop=True, NER_tags='B-ING'),
  Pandas(sent_id=0, token_text='wheat', token_lemma='wheat', token_pos='NOUN', token_tag='NN', token_dep='compound', token_shape='xxxx', token_is_alpha=True, token_is_stop=False, NER_tags='I-ING'),
  Pandas(sent_id=0, token_text='bread', token_lemma='bread', token_pos='NOUN', token_tag='NN', token_dep='ROOT', token_shape='xxxx', token_is_alpha=True, token_is_stop=False, NER_tags='I-ING')],
 [Pandas(sent_id=1, token_text

andas(sent_id=0, token_text='1', token_lemma='1', token_pos='NUM', token_tag='CD', token_dep='nummod', token_shape='d', token_is_alpha=False, token_is_stop=False, NER_tags='B-QUANTITY')

In [6]:
# input is a sentence as a structure show above 
#and and ith word from the sentence to return the features for that word

# Sample training data (replace with your actual data)

def word2features(sent, i):
    word = str(sent[i].token_text)
    postag = sent[i].token_tag
    
    # data structure consisting of a feature name and value for the token
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(), # lower case variant of the token
        'word[-3:]': word[-3:], #suffix of 3 characters
        'word[-2:]': word[-2:], #suffix of 2 characters
        'word.isupper()': word.isupper(), # initial captial
        'word.istitle()': word.istitle(), # all words ini caps
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2], #first two characters of the PoS Tag
    }
    if i > 0:
        # adding features for the word based on the previous word
        word1 = str(sent[i-1].token_text) # previous word
        postag1 = sent[i-1].token_tag
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True # Beginning of sentence as a feature

    if i < len(sent)-1:
        # adding features for the word based on the next word
        word1 = str(sent[i+1].token_text) # next word
        postag1 = sent[i+1].token_tag
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True # end of sentence as a feature

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [x.NER_tags for x in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [7]:
X_train = [sent2features(s) for s in sents[:144000]]
y_train = [sent2labels(s) for s in sents[:144000]]

X_test = [sent2features(s) for s in sents[144000:]]
y_test = [sent2labels(s) for s in sents[144000:]]

# X_train = [sent2features(s) for s in sents[:1000]]
# y_train = [sent2labels(s) for s in sents[:1000]]

# X_test = [sent2features(s) for s in sents[1000:]]
# y_test = [sent2labels(s) for s in sents[1000:]]

In [8]:
y_train[0]

['B-QUANTITY', 'B-UNIT', 'B-ING', 'I-ING', 'I-ING']

In [9]:
import sklearn_crfsuite

from sklearn_crfsuite import CRF

# different parameters are used for training
# check https://sklearn-crfsuite.readthedocs.io/en/latest/api.html?highlight=CRF
crf = CRF(algorithm='lbfgs',
          c1=0.1, #The coefficient for L1 regularization.
          c2=0.1, #The coefficient for L2 regularization.
          max_iterations=100,
          all_possible_transitions=False)

In [10]:
crf.fit(X=X_train, y=y_train)

CRF(algorithm='lbfgs', all_possible_transitions=False, c1=0.1, c2=0.1,
    max_iterations=100)

In [19]:
import pickle

with open('crf_big.pkl', 'wb') as f:
    pickle.dump(crf, f)

In [11]:
from sklearn_crfsuite.metrics import flat_classification_report
pred = crf.predict(X_test)
report = flat_classification_report(y_pred=pred, y_true=y_test)

In [12]:
print(report)

              precision    recall  f1-score   support

       B-ING       0.97      0.97      0.97     43741
   B-PRODUCT       0.95      0.96      0.95      3581
  B-QUANTITY       1.00      1.00      1.00     43778
     B-STATE       0.97      0.96      0.97     28151
      B-UNIT       1.00      1.00      1.00     36395
       I-ING       0.96      0.97      0.96     50024
   I-PRODUCT       0.93      0.96      0.94     11549
  I-QUANTITY       0.99      0.99      0.99       359
     I-STATE       0.95      0.96      0.96     27016
      I-UNIT       1.00      1.00      1.00       503
           O       0.95      0.93      0.94     45346

    accuracy                           0.97    290443
   macro avg       0.97      0.97      0.97    290443
weighted avg       0.97      0.97      0.97    290443



In [13]:
from collections import Counter

Counter([x.NER_tags for y in sents[:144000] for x in y])

Counter({'I-ING': 190411,
         'O': 174954,
         'B-ING': 166340,
         'B-QUANTITY': 166088,
         'B-UNIT': 138598,
         'B-STATE': 107162,
         'I-STATE': 103138,
         'I-PRODUCT': 43663,
         'B-PRODUCT': 13744,
         'I-UNIT': 1850,
         'I-QUANTITY': 1560})

In [13]:
def word2featuresmod(sent, i):
    word = sent[i].text
    postag = sent[i].tag_
    
    # data structure consisting of a feature name and value for the token
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(), # lower case variant of the token
        'word[-3:]': word[-3:], #suffix of 3 characters
        'word[-2:]': word[-2:], #suffix of 2 characters
        'word.isupper()': word.isupper(), # initial captial
        'word.istitle()': word.istitle(), # all words ini caps
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2], #first two characters of the PoS Tag
    }
    if i > 0:
        # adding features for the word based on the previous word
        word1 = sent[i-1].text # previous word
        postag1 = sent[i-1].tag_
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True # Beginning of sentence as a feature

    if i < len(sent)-1:
        # adding features for the word based on the next word
        word1 = sent[i+1].text # next word
        postag1 = sent[i+1].tag_
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True # end of sentence as a feature

    return features

def sent2featuresmod(sent):
    return [word2featuresmod(sent, i) for i in range(len(sent))]

def sent2labelsmod(sent):
    return [x.NER_tags for x in sent]

def sent2tokensmod(sent):
    return [token for token, postag, label in sent]

In [16]:
import spacy

nlp = spacy.load("en_core_web_sm")

text = "6 ounces blue cheese,at room temperature"

doc = nlp(text)

tokens = [token for token in doc]

In [17]:
prep = sent2featuresmod(tokens)
# prep

predict = crf.predict([prep])


In [18]:
predict

[['B-QUANTITY', 'B-UNIT', 'B-ING', 'I-ING', 'O', 'O', 'O', 'O']]

In [16]:
df_ar_gk = pd.read_csv('ar_gk_train_cleaned.csv')

In [17]:
df_ar_gk

,sent_id,token_text,token_lemma,token_tag,NER_tags
0,0,4,4,CD,B-QUANTITY
1,0,cloves,clove,NNS,B-UNIT
2,0,garlic,garlic,JJ,B-NAME
3,1,2,2,CD,B-QUANTITY
4,1,tablespoons,tablespoon,NNS,B-UNIT
...,...,...,...,...,...
37209,6609,-RRB-,-RRB-,NN,O
37210,6610,1 1/4,1 1/4,CD,B-QUANTITY
37211,6610,teaspoons,teaspoon,NNS,B-UNIT
37212,6610,baking,baking,VBG,B-NAME


In [18]:
sents = []
temp = []
tags = []
temp_tags = []
current_id = 0

for item in df_ar_gk.itertuples(index=False):
    if item.sent_id != current_id:
        current_id += 1
        sents.append(temp)
        tags.append(temp_tags)
        temp = []
        temp_tags = []
    temp.append(item)
    temp_tags.append(item.NER_tags)


In [19]:
sents[:2]

[[Pandas(sent_id=0, token_text='4', token_lemma='4', token_tag='CD', NER_tags='B-QUANTITY'),
  Pandas(sent_id=0, token_text='cloves', token_lemma='clove', token_tag='NNS', NER_tags='B-UNIT'),
  Pandas(sent_id=0, token_text='garlic', token_lemma='garlic', token_tag='JJ', NER_tags='B-NAME')],
 [Pandas(sent_id=1, token_text='2', token_lemma='2', token_tag='CD', NER_tags='B-QUANTITY'),
  Pandas(sent_id=1, token_text='tablespoons', token_lemma='tablespoon', token_tag='NNS', NER_tags='B-UNIT'),
  Pandas(sent_id=1, token_text='vegetable', token_lemma='vegetable', token_tag='JJ', NER_tags='B-NAME'),
  Pandas(sent_id=1, token_text='oil', token_lemma='oil', token_tag='NN', NER_tags='I-NAME'),
  Pandas(sent_id=1, token_text=',', token_lemma=',', token_tag=',', NER_tags='O'),
  Pandas(sent_id=1, token_text='divided', token_lemma='divided', token_tag='VBD', NER_tags='B-STATE')]]

In [20]:
X_train = [sent2features(s) for s in sents]
y_train = [sent2labels(s) for s in sents]

# X_test = [sent2features(s) for s in sents[144000:]]
# y_test = [sent2labels(s) for s in sents[144000:]]

In [21]:
import sklearn_crfsuite

from sklearn_crfsuite import CRF

# different parameters are used for training
# check https://sklearn-crfsuite.readthedocs.io/en/latest/api.html?highlight=CRF
crf = CRF(algorithm='lbfgs',
          c1=0.1, #The coefficient for L1 regularization.
          c2=0.1, #The coefficient for L2 regularization.
          max_iterations=100,
          all_possible_transitions=False)

In [22]:
crf.fit(X=X_train, y=y_train)

CRF(algorithm='lbfgs', all_possible_transitions=False, c1=0.1, c2=0.1,
    max_iterations=100)

In [23]:
df_ar_gk_test = pd.read_csv('ar_gk_test_cleaned.csv')

sents_test = []
temp = []
tags_test = []
temp_tags = []
current_id = 0

for item in df_ar_gk_test.itertuples(index=False):
    if item.sent_id != current_id:
        current_id += 1
        sents_test.append(temp)
        tags_test.append(temp_tags)
        temp = []
        temp_tags = []
    temp.append(item)
    temp_tags.append(item.NER_tags)

In [24]:
sents_test[:2]

[[Pandas(sent_id=0, token_text='1/2', token_lemma='1/2', token_tag='CD', NER_tags='B-QUANTITY'),
  Pandas(sent_id=0, token_text='large', token_lemma='large', token_tag='JJ', NER_tags='SIZE'),
  Pandas(sent_id=0, token_text='sweet', token_lemma='sweet', token_tag='JJ', NER_tags='B-NAME'),
  Pandas(sent_id=0, token_text='red', token_lemma='red', token_tag='JJ', NER_tags='I-NAME'),
  Pandas(sent_id=0, token_text='onion', token_lemma='onion', token_tag='NN', NER_tags='I-NAME'),
  Pandas(sent_id=0, token_text=',', token_lemma=',', token_tag=',', NER_tags='O'),
  Pandas(sent_id=0, token_text='thinly', token_lemma='thinly', token_tag='RB', NER_tags='O'),
  Pandas(sent_id=0, token_text='sliced', token_lemma='sliced', token_tag='VBD', NER_tags='B-STATE')],
 [Pandas(sent_id=1, token_text='6', token_lemma='6', token_tag='CD', NER_tags='B-QUANTITY'),
  Pandas(sent_id=1, token_text='ounces', token_lemma='ounce', token_tag='NNS', NER_tags='B-UNIT'),
  Pandas(sent_id=1, token_text='fresh', token_lemm

In [25]:
X_test = [sent2features(s) for s in sents_test]
y_test = [sent2labels(s) for s in sents_test]

In [26]:
from sklearn_crfsuite.metrics import flat_classification_report
pred = crf.predict(X_test)
report = flat_classification_report(y_pred=pred, y_true=y_test)
print(report)

              precision    recall  f1-score   support

      B-NAME       0.91      0.93      0.92      2130
  B-QUANTITY       0.98      0.99      0.99      2086
     B-STATE       0.89      0.94      0.91       943
      B-UNIT       0.97      0.97      0.97      1725
          DF       0.96      0.96      0.96       196
      I-NAME       0.89      0.90      0.89      1512
  I-QUANTITY       0.95      0.86      0.90        65
     I-STATE       0.81      0.68      0.74       150
      I-UNIT       0.90      0.90      0.90        50
           O       0.93      0.90      0.92      3655
        SIZE       0.97      0.97      0.97       104
        TEMP       0.84      0.84      0.84        43

    accuracy                           0.93     12659
   macro avg       0.92      0.90      0.91     12659
weighted avg       0.93      0.93      0.93     12659



In [ ]:
              precision    recall  f1-score   support

       B-ING       0.86      0.87      0.86    208934
   B-PRODUCT       0.76      0.59      0.66     17216
  B-QUANTITY       0.99      1.00      1.00    208712
     B-STATE       0.90      0.90      0.90    134547
      B-UNIT       0.97      0.98      0.97    174033
       I-ING       0.82      0.90      0.86    239159
   I-PRODUCT       0.67      0.62      0.64     54871
  I-QUANTITY       1.00      0.95      0.97      1906
     I-STATE       0.89      0.88      0.89    129419
      I-UNIT       1.00      0.99      1.00      2339
           O       0.93      0.86      0.89    219008

    accuracy                           0.90   1390144
   macro avg       0.89      0.87      0.88   1390144
weighted avg       0.90      0.90      0.90   1390144

